In [9]:
import ast
import inspect

class VariableNameChecker(ast.NodeVisitor):
    def __init__(self):
        self.var_nodes = []         # Stores assignment nodes
        self.assigned_vars = set()  # Stores names of assigned variables
        self.used_vars = set()      # Stores names of used variables

    def visit_Assign(self, node):
        for target in node.targets:
            if isinstance(target, ast.Name):
                self.var_nodes.append(node)
                self.assigned_vars.add(target.id)
        self.generic_visit(node)

    def visit_Name(self, node):
        if isinstance(node.ctx, ast.Load):
            self.used_vars.add(node.id)
        self.generic_visit(node)

    def print_vars(self):
        print("Printing Detected Variables and Their Values:")
        for var_node in self.var_nodes:
            out_string = f"{var_node.targets[0].id} = "

            # Check the type of the value and print accordingly
            value = var_node.value
            if isinstance(value, ast.Name):
                out_string += str(value.id)
            elif isinstance(value, ast.Constant):
                out_string += str(value.value)
            elif isinstance(value, ast.BinOp):
                # Recursively get the string representation of the BinOp
                out_string += self.get_binop_string(value)
            elif isinstance(value, ast.Call):
                out_string += self.get_call_string(value)
            elif isinstance(value, ast.UnaryOp):
                out_string += self.get_unaryop_string(value)
            else:
                out_string += "<complex expression>"

            print(out_string)

    def get_binop_string(self, node):
        left = self.get_node_string(node.left)
        right = self.get_node_string(node.right)
        op = self.get_op_symbol(node.op)
        return f"({left} {op} {right})"

    def get_call_string(self, node):
        func_name = node.func.id
        args = ', '.join([self.get_node_string(arg) for arg in node.args])
        return f"{func_name}({args})"

    def get_unaryop_string(self, node):
        operand = self.get_node_string(node.operand)
        op = self.get_op_symbol(node.op)
        return f"{op}{operand}"

    def get_node_string(self, node):
        if isinstance(node, ast.Name):
            return node.id
        elif isinstance(node, ast.Constant):
            return str(node.value)
        elif isinstance(node, ast.BinOp):
            return self.get_binop_string(node)
        elif isinstance(node, ast.Call):
            return self.get_call_string(node)
        elif isinstance(node, ast.UnaryOp):
            return self.get_unaryop_string(node)
        else:
            return "<expr>"

    def get_op_symbol(self, op):
        # Map AST operator classes to their symbols
        operators = {
            ast.Add: '+',
            ast.Sub: '-',
            ast.Mult: '*',
            ast.Div: '/',
            ast.FloorDiv: '//',
            ast.Mod: '%',
            ast.Pow: '**',
            ast.LShift: '<<',
            ast.RShift: '>>',
            ast.BitOr: '|',
            ast.BitXor: '^',
            ast.BitAnd: '&',
            ast.MatMult: '@',
            ast.USub: '-',
            ast.UAdd: '+',
        }
        return operators.get(type(op), '?')

class UnusedVarChecker:
    def __init__(self):
        self.unused_vars = set()

    def find_unused_vars(self, assigned_vars, used_vars):
        self.unused_vars = assigned_vars - used_vars
        if not self.unused_vars:
            print("\nUnusedVarChecker: No unused variables found.")
        else:
            print("\nUnusedVarChecker: Unused variables detected:", self.unused_vars)
            for var in self.unused_vars:
                print(f"Variable '{var}' is assigned but never used.")


class Linter(VariableNameChecker, UnusedVarChecker):
    def __init__(self):
        VariableNameChecker.__init__(self)
        UnusedVarChecker.__init__(self)



In [10]:
def binary_search(arr, target):
    low = 0
    high = len(arr) - 1
    mid = -1 
    x=0
     # Assigned but may not be used directly outside the loop

    while low <= high:
        mid = (low + high) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            low = mid + 1
        else:
            high = mid - 1
    return -1


In [11]:
def main():
    # Get source code for binary_search function
    source_code = inspect.getsource(binary_search)
    ast_tree = ast.parse(source_code)

    # Print the AST structure (optional)
    print("AST tree:\n", ast.dump(ast_tree, indent=4))

    # Initialize the linter
    linter = Linter()

    # Visit the AST nodes
    linter.visit(ast_tree)

    # Print variables and their assigned values
    linter.print_vars()

    # Find and report unused variables
    linter.find_unused_vars(linter.assigned_vars, linter.used_vars)

main()


AST tree:
 Module(
    body=[
        FunctionDef(
            name='binary_search',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='arr'),
                    arg(arg='target')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Assign(
                    targets=[
                        Name(id='low', ctx=Store())],
                    value=Constant(value=0)),
                Assign(
                    targets=[
                        Name(id='high', ctx=Store())],
                    value=BinOp(
                        left=Call(
                            func=Name(id='len', ctx=Load()),
                            args=[
                                Name(id='arr', ctx=Load())],
                            keywords=[]),
                        op=Sub(),
                        right=Constant(value=1))),
                Assign(
     